## Simple Mission

This notebook serves to check the ardupilot installation

In [1]:
import os
import subprocess
import sys
from pathlib import Path
from typing import cast

import pymavlink.dialects.v20.ardupilotmega as mavlink

root = Path.cwd().resolve().parents[1]
sys.path.append(str(root))  

from config import ARDUPILOT_VEHICLE_PATH, LOGS_PATH, VEH_PARAMS_PATH  # noqa: E402
from helpers import clean  # noqa: E402
from mavlink.customtypes.mission import MissionLoader  # noqa: E402
from mavlink.enums import Cmd, CmdNav, MissionResult, MsgID  # noqa: E402
from mavlink.util import ask_msg, connect, stop_msg  # noqa: E402
from plan import State  # noqa: E402
from plan.planner import Plan  # noqa: E402

clean()

## Launch Copter (ardupilot)

In [2]:
#This must agrre with first waypoint in mission.waypoint
spawn_str= '-35.3633245,149.1652241,0.0,0' 

sim_vehicle_path = os.path.expanduser(ARDUPILOT_VEHICLE_PATH)
vehicle_cmd = (
    f"python3 {sim_vehicle_path} "
    f"-v ArduCopter "
    f"-I0 --sysid 1 "
    f"--no-rebuild "
    f"--use-dir={LOGS_PATH} "
    f"--add-param-file {VEH_PARAMS_PATH} "
    f"--no-mavproxy "
    f'--custom-location={spawn_str}'
)


# Open a new terminal and run the command
subprocess.Popen(["gnome-terminal","--title=planner","--", 
                  "bash", "-c", f"{vehicle_cmd}; exec bash"])

<Popen: returncode: None args: ['gnome-terminal', '--title=planner', '--', '...>

## 1. Connect to the vehicle

In [3]:
conn = connect("tcp:127.0.0.1:5760")
conn.wait_heartbeat() 
print("✅ Heartbeat received")


[Errno 111] Connection refused sleeping
✅ Heartbeat received


# 2. Load the waypoint file

In [4]:
mission = MissionLoader(conn.target_system, conn.target_component)
mission_path = (root / "plan" / "missions" / "simple_mission.waypoints").resolve()
count = mission.load(str(mission_path)) # your QGC WPL 110 file
print(f"✅ Loaded {count} waypoints")

✅ Loaded 4 waypoints


# 3. Read the mission

In [5]:
# Retrieve each mission item
for i in range(count):
    wp = mission.item(i)
    cmd_name = CmdNav(wp.command).name
    print(
        f"🧭 Mission[{i}] → cmd: {cmd_name}, "
        f"x: {wp.x}, y: {wp.y}, z: {wp.z}, current: {wp.current}"
    )

🧭 Mission[0] → cmd: WAYPOINT, x: -35.3633245, y: 149.1652241, z: 0.0, current: 0
🧭 Mission[1] → cmd: TAKEOFF, x: -35.3633245, y: 149.1652241, z: 5.0, current: 0
🧭 Mission[2] → cmd: WAYPOINT, x: -35.3632795, y: 149.1652241, z: 5.0, current: 0
🧭 Mission[3] → cmd: LAND, x: -35.3632795, y: 149.1652241, z: 0.0, current: 0


# 3. Clear any existing mission

In [6]:
conn.mav.mission_clear_all_send(conn.target_system, conn.target_component)
while True:
    msg = conn.recv_match(type='STATUSTEXT',blocking=False)
    if msg and msg.text == 'ArduPilot Ready':
        print("🧹 Cleared previous mission")
        break



🧹 Cleared previous mission


# 4. Send new mission count

In [7]:
conn.mav.mission_count_send(conn.target_system, conn.target_component, mission.count())
print(f"📤 Sending {mission.count()} mission items")


📤 Sending 4 mission items


# 5. Send each item in response to MISSION_REQUEST

In [8]:
for i in range(mission.count()):
    msg = conn.recv_match(type='MISSION_REQUEST', blocking=True, timeout=5)
    if not msg or msg.seq != i:
        raise RuntimeError(f"❌ Unexpected mission request: {msg}")
    conn.mav.send(mission.wp(i))
    print(f"✅ Sent mission item {i}")

✅ Sent mission item 0
✅ Sent mission item 1
✅ Sent mission item 2
✅ Sent mission item 3


# 6. Wait for final MISSION_ACK

In [9]:
ack = conn.recv_match(type='MISSION_ACK', blocking=True, timeout=5)
if ack and MissionResult(ack.type) == MissionResult.ACCEPTED:
    print("🎉 Mission upload successful!")
else:
    print(f"⚠️ Mission upload failed or timed out: {ack}")

🎉 Mission upload successful!


## Check the mission

In [10]:
# Create a mission protocol loader
wp = MissionLoader()


# Request list of mission items
conn.mav.mission_request_list_send(conn.target_system, conn.target_component)

# Wait for mission count
msg = conn.recv_match(type='MISSION_COUNT', blocking=True, timeout=5)
if msg:
    mission_count = msg.count
    print(f"📦 UAV has {mission_count} mission items")
    # Retrieve each mission item
    for i in range(mission_count):
        conn.mav.mission_request_send(conn.target_system, conn.target_component, i)
        item = conn.recv_match(type='MISSION_ITEM', blocking=True, timeout=5)
        if item:
            wp.set(item, i)
            print(
                f"🧭 Mission[{i}] → cmd: {CmdNav(item.command).name}, "
                f"x: {item.x}, y: {item.y}, z: {item.z}, current: {item.current}"
            )
else:
    print('No mission count received')

📦 UAV has 4 mission items
🧭 Mission[0] → cmd: WAYPOINT, x: 0.0, y: 0.0, z: 0.0, current: 0
🧭 Mission[1] → cmd: TAKEOFF, x: -35.36332702636719, y: 149.16522216796875, z: 5.0, current: 0
🧭 Mission[2] → cmd: WAYPOINT, x: -35.36328125, y: 149.16522216796875, z: 5.0, current: 0
🧭 Mission[3] → cmd: LAND, x: -35.36328125, y: 149.16522216796875, z: 0.0, current: 0


# Arm the uav

In [11]:
plan_auto = Plan.arm(name="arm")
plan_auto.bind(conn,verbose=2)
while plan_auto.state != State.DONE:
    plan_auto.act()

Vehicle 1: ▶️ Plan Started: 📋 arm
Vehicle 1: ▶️ Action Started: 🔧 PREARM
Vehicle 1: ▶️ Step Started: Check disarmed
Vehicle 1: ✅ Step Done: Check disarmed
Vehicle 1: ▶️ Step Started: Check EKF status
Vehicle 1: ✅ Step Done: Check EKF status
Vehicle 1: ▶️ Step Started: Check GPS
Vehicle 1: ✅ Step Done: Check GPS
Vehicle 1: ▶️ Step Started: Check system
Vehicle 1: ✅ Step Done: Check system
Vehicle 1: ✅ Action Done: 🔧 PREARM
Vehicle 1: ▶️ Action Started: ⚙️ MODE: GUIDED
Vehicle 1: ▶️ Step Started: Switch to GUIDED
Vehicle 1: ✅ Step Done: Switch to GUIDED
Vehicle 1: ✅ Action Done: ⚙️ MODE: GUIDED
Vehicle 1: ▶️ Action Started: 🔐 ARM
Vehicle 1: ▶️ Step Started: arm
Vehicle 1: ✅ Step Done: arm
Vehicle 1: ✅ Action Done: 🔐 ARM
Vehicle 1: ✅ Plan Done: 📋 arm


# Check Sensors

In [12]:
conn.mav.request_data_stream_send(
    conn.target_system,               # target_system
    conn.target_component,            # target_component
    mavutil.mavlink.MAV_DATA_STREAM_ALL,  # req_stream_id
    5,                                # req_message_rate (Hz)
    1                                 # start_stop (1 = start)
)


NameError: name 'mavutil' is not defined

In [ ]:
from pymavlink import mavutil

def check_sensor_messages(conn):
    print("🔍 Checking for active sensor MAVLink messages...")

    # Define MAVLink message types associated with known sensors
    sensor_msgs = {
        "RAW_IMU": "IMU (accelerometer + gyro)",
        "HIGHRES_IMU": "High-resolution IMU",
        "SCALED_PRESSURE": "Barometer",
        "GPS_RAW_INT": "GPS",
        "GPS2_RAW": "Secondary GPS",
        "RAW_MAG": "Compass (magnetometer)",
        "OPTICAL_FLOW_RAD": "Optical Flow",
        "DISTANCE_SENSOR": "Proximity / RangeFinder",
        "OBSTACLE_DISTANCE": "Obstacle Avoidance",
        "ADSB_VEHICLE": "ADSB Receiver",
        "VISION_POSITION_ESTIMATE": "Vision/VIO",
        "IRLOCK_REPORT": "IRLock Landing Beacon",
        "VICON_POSITION_ESTIMATE": "Motion Capture",
        "UWB_DISTANCE": "UWB Beacon",
    }

    found = set()

    # Listen for messages for a few seconds
    conn.mav.request_data_stream_send(
        conn.target_system, conn.target_component,
        mavutil.mavlink.MAV_DATA_STREAM_ALL,
        4, 1
    )

    print("Waiting for sensor messages (5 seconds)...")
    import time
    start = time.time()
    while time.time() - start < 5:
        msg = conn.recv_match(blocking=False)
        if msg:
            msg_type = msg.get_type()
            if msg_type in sensor_msgs and msg_type not in found:
                print(f"✅ {sensor_msgs[msg_type]}: {msg_type}")
                found.add(msg_type)

    if not found:
        print("⚠️  No sensor messages received in 5 seconds. Is the drone armed and sensors active?")

def check_sensor_params(conn):
    print("\n🔍 Checking for sensor-related parameters...")

    sensor_params = {
        "RAW_IMU": "Basic IMU: Accelerometer + Gyroscope",
        "SCALED_IMU2": "Second IMU (if present)",
        "SCALED_IMU3": "Third IMU (if present)",
        "HIGHRES_IMU": "High-resolution IMU (full sensor data at high rate)",
        "SCALED_PRESSURE": "Barometer (altitude estimation)",
        "GPS_RAW_INT": "GPS primary",
        "GPS2_RAW": "GPS secondary",
        "RAW_MAG": "Magnetometer (Compass)",
        "DISTANCE_SENSOR": "Rangefinder / Sonar / Lidar",
        "OPTICAL_FLOW_RAD": "Optical Flow (PX4Flow, etc.)",
        "OBSTACLE_DISTANCE": "Obstacle Avoidance grid",
        "ADSB_VEHICLE": "ADSB aircraft data (e.g., for traffic avoidance)",
        "VISION_POSITION_ESTIMATE": "Vision-based pose estimate (e.g., VIO)",
        "VISION_SPEED_ESTIMATE": "Vision-based velocity estimate",
        "IRLOCK_REPORT": "IRLock beacon (for precision landing)",
        "VICON_POSITION_ESTIMATE": "Motion capture system (VICON)",
        "UWB_DISTANCE": "Ultra-wideband beacon distance (e.g., Pozyx)",
    }

    for param, description in sensor_params.items():
        try:
            value = conn.param_fetch_one(param)
            print(f"✅ {description}: {value}")
        except:
            print(f"⚠️  {description} not found")
check_sensor_messages(conn)
check_sensor_params(conn)


🔍 Checking for active sensor MAVLink messages...
Waiting for sensor messages (5 seconds)...
✅ IMU (accelerometer + gyro): RAW_IMU
✅ Barometer: SCALED_PRESSURE
✅ GPS: GPS_RAW_INT

🔍 Checking for sensor-related parameters...
✅ Basic IMU: Accelerometer + Gyroscope: None
✅ Second IMU (if present): None
✅ Third IMU (if present): None
✅ High-resolution IMU (full sensor data at high rate): None
✅ Barometer (altitude estimation): None
✅ GPS primary: None
✅ GPS secondary: None
✅ Magnetometer (Compass): None
✅ Rangefinder / Sonar / Lidar: None
✅ Optical Flow (PX4Flow, etc.): None
✅ Obstacle Avoidance grid: None
✅ ADSB aircraft data (e.g., for traffic avoidance): None
✅ Vision-based pose estimate (e.g., VIO): None
✅ Vision-based velocity estimate: None
✅ IRLock beacon (for precision landing): None
✅ Motion capture system (VICON): None
✅ Ultra-wideband beacon distance (e.g., Pozyx): None


In [ ]:
import os
import csv
import time
from pymavlink import mavutil

# Output folder
output_dir = "sensor_logs"
os.makedirs(output_dir, exist_ok=True)

# Request data stream (ensures ArduPilot sends sensor data continuously)
conn.mav.request_data_stream_send(
    conn.target_system,
    conn.target_component,
    mavutil.mavlink.MAV_DATA_STREAM_ALL,
    5,  # 5 Hz
    1   # Start sending
)

# Define sensor message types and CSV writers
sensor_types = {
    "RAW_IMU": None,
    "SCALED_PRESSURE": None,
    "GPS_RAW_INT": None,
}

# Prepare open file handles and CSV writers
writers = {}
files = {}

for msg_type in sensor_types:
    filename = os.path.join(output_dir, f"{msg_type}.csv")
    f = open(filename, "w", newline="")
    files[msg_type] = f
    writers[msg_type] = csv.writer(f)
    writers[msg_type].writerow(["timestamp", "field", "value"])  # Generic header

print("📡 Logging sensor messages... Press Ctrl+C to stop.\n")

try:
    while True:
        msg = conn.recv_match(type=list(sensor_types.keys()), blocking=True, timeout=10)
        if msg:
            msg_type = msg.get_type()
            timestamp = time.time()
            for field_name in msg.__dict__:
                if not field_name.startswith("_"):
                    value = getattr(msg, field_name)
                    writers[msg_type].writerow([timestamp, field_name, value])
except KeyboardInterrupt:
    print("\n🛑 Logging stopped by user.")
finally:
    for f in files.values():
        f.close()
